# 大模型归一化

## 1、BatchNorm  
在大模型中一般不进行使用, 传统的 BatchNorm（批归一化）是为图像数据设计的（如 (B, C, H, W)
BatchNorm是针对每一个样本，每一层进行归一化，其公式如下：  
$$
\hat{x} = \frac{x - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}
$$
$$
y = \gamma \hat{x} + \beta
$$
其中，$\mu_B$和$\sigma_B^2$是当前batch的均值和方差，$\epsilon$是为了防止除零操作，$\gamma$和$\beta$是可学习的参数，用于对归一化后的数据进行缩放和平移。  
  
BatchNorm在训练和测试阶段的行为是不同的，训练阶段会计算整个batch的均值和方差，测试阶段会使用训练阶段的均值和方差。

[B, C, H, W]四个维度为对C维度进行作用，与B, H, W无关，共计算了C个均值和方差。
  
视频：https://www.bilibili.com/video/BV12d4y1f74C/?spm_id_from=333.337.search-card.all.click&vd_source=071b23b9c7175dbaf674c65294124341

## 2、层归一化（LayerNorm）  
层归一化是针对每一个样本，每一层进行归一化，其公式如下：  
$$
\hat{x} = \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}}
$$
$$
y = \gamma \hat{x} + \beta
$$
其中，$\mu$和$\sigma^2$是当前层的均值和方差，$\epsilon$是为了防止除零操作，$\gamma$和$\beta$是可学习的参数，用于对归一化后的数据进行缩放和平移。  
  
(B, T, C)三个维度为对C维度进行作用，与B, T无关，共计算了B*T个均值和方差。  
  
视频：https://www.bilibili.com/video/BV1yCyQY6EnA/?spm_id_from=333.337.search-card.all.click&vd_source=071b23b9c7175dbaf674c65294124341

## 3、RMSNorm  
RMSNorm是针对每一个样本，每一层进行归一化，其公式如下：  
$$
\hat{x} = \frac{x}{\sqrt{\sigma^2 + \epsilon}}
$$
RMS(x)表示x的均方根，即$\sigma^2$，即$\sqrt{\frac{\sum_{i=1}^n x_i^2}{n}}$。在C上计算，与B, T无关，共计算了B*T个RMS值。
$$
y = \gamma \hat{x} + \beta
$$
其中，$\sigma^2$是当前层的平方和，$\epsilon$是为了防止除零操作。  
RMSNorm和layer_norm的主要区别在于RMSNorm不需要同时计算均值和方差两个统计量，而只需要计算均方根这一个统计量。在模型表现效果几乎与layer_norm持平的前提下，节省7%-64%的计算量。

In [ ]:
import torch
import torch.nn as nn

# 定义RMSNorm类
class MyRMSNorm(nn.Module):
    def __init__(self, normalized_shape, epsilon=1e-8):
        super(MyRMSNorm, self).__init__()
        self.epsilon = epsilon
        self.gamma = nn.Parameter(torch.ones(normalized_shape))  # 缩放参数

    def forward(self, x):
        '''
        x.shape: (B, T, C)
        x**2.shape: (B, T, C)
        torch.mean(x**2, dim=-1, keepdim=True).shape: (B, T, 1)，即为RMS(x)的值
        rms.shape: (B, T, 1)
        '''
        rms = torch.sqrt(torch.mean(x**2, dim=-1, keepdim=True) + self.epsilon)  # 计算RMS值
        x = x / rms  # 归一化
        return self.gamma * x  # 缩放


# class QuickRMSNorm(nn.Module):
#     def __init__(self, layer_shape, eps=1e-8, bias=False):
#         super(QuickRMSNorm, self).__init__()

#         # torch中register_parameter()功能为：向我们建立的网络module添加parameter
#         # 因此，我们需要对pytorch官方封装好的RMSNorm功能模块添加一个可以训练参数的层，命名为scale，并初始化为形状为layer_shape，所有值为1的张量矩阵。
#         self.register_parameter("scale", nn.Parameter(torch.ones(layer_shape)))

#     def forward(self, x):
#         # 计算Frobenius范数（球某个矩阵中所有元素的平方和再开方得到，该范数用来衡量矩阵的大小，详情请百度）, RMS = 1/sqrt(N) * Frobenius
#         # 具体来说，torch.linalg.norm(x, dim=(1, 2))计算了x在第1和第2维度上的范数。然后，将结果乘以x[0].numel() ** -.5。x[0].numel()表示x第一个元素（即x的第一行）的元素个数，** -.5表示求平方根的倒数。
#         ff_rms = torch.linalg.norm(x, dim=(1,2)) * x[0].numel() ** -.5
#         # print(ff_rms.shape)
#         # 将ff_rms算子应用于输入的张量x，依据公式，做除法，因为输入向量x是三维的，因此需要对ff_rms进行升两维，也变成三维的张量。这样可以进行元素之间的计算。
#         raw = x / ff_rms.unsqueeze(-1).unsqueeze(-1)
#         # print(raw.shape)

#         # 返回缩放后归一化的张量
#         # print(self.scale[:x.shape[1], :].unsqueeze(0) * raw)
#         return self.scale[:x.shape[1], :].unsqueeze(0) * raw

# 创建一个输入张量
input_tensor = torch.tensor([[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]], dtype=torch.float32)
print("原始输入张量:")
print(input_tensor.shape)
print(input_tensor)

# 1、使用自定义的RMSNorm
# 创建RMSNorm实例
normalized_shape = input_tensor.shape[-1]  # 归一化的维度
rms_norm = MyRMSNorm(normalized_shape)
# 对输入张量进行RMSNorm归一化
normalized_tensor = rms_norm(input_tensor)
print("\n1、使用自定义的RMSNorm归一化后的张量:")
print(normalized_tensor)

# # 2、使用快速RMSNorm
# quick_norm = QuickRMSNorm((input_tensor.shape[1],input_tensor.shape[2]))
# # 对输入张量进行RMSNorm归一化
# normalized_tensor = quick_norm(input_tensor)
# print("\n3、2、使用快速RMSNorm归一化后的张量:")
# print(normalized_tensor)


# 3、使用pytorch自带的RMSNorm
layer_norm = nn.RMSNorm(normalized_shape)
# 对输入张量进行RMSNorm归一化
normalized_tensor = layer_norm(input_tensor)
print("\n3、使用pytorch自带的RMSNorm归一化后的张量:")
print(normalized_tensor)

原始输入张量:
torch.Size([1, 2, 3])
tensor([[[1., 2., 3.],
         [4., 5., 6.]]])

1、使用自定义的RMSNorm归一化后的张量:
tensor([[[0.4629, 0.9258, 1.3887],
         [0.7895, 0.9869, 1.1843]]], grad_fn=<MulBackward0>)

3、2、使用快速RMSNorm归一化后的张量:
tensor([[[0.2568, 0.5136, 0.7703],
         [1.0271, 1.2839, 1.5407]]], grad_fn=<MulBackward0>)

3、使用pytorch自带的RMSNorm归一化后的张量:
tensor([[[0.4629, 0.9258, 1.3887],
         [0.7895, 0.9869, 1.1843]]], grad_fn=<MulBackward0>)
